In [1]:
import pandas as pd
import numpy as np

In [2]:
gender = pd.read_csv('D:/Data science/DatingRecommendationSystem/libimseti/gender.dat',names = ['userid','gender'],header=None)

print(gender.head())

print(len(gender))

   userid gender
0       1      F
1       2      F
2       3      U
3       4      F
4       5      F
220970


In [3]:
ratings = pd.read_csv('D:/Data science/DatingRecommendationSystem/libimseti/ratings.dat',names=['userid','profileid','rating'],header=None)

print(ratings.head())

print(len(ratings))

   userid  profileid  rating
0       1        133       8
1       1        720       6
2       1        971      10
3       1       1095       7
4       1       1616      10
17359346


In [4]:
print('userid')
print('min id: ',min(ratings['userid']))
print('max id: ',max(ratings['userid']))
print('\n')

print('profileid')
print('min id: ',min(ratings['profileid']))
print('max id: ',max(ratings['profileid']))

userid
min id:  1
max id:  135359


profileid
min id:  1
max id:  220970


In [5]:
merged = ratings.merge(gender,on='userid',how='left')

print(merged.head())
print(len(merged))

   userid  profileid  rating gender
0       1        133       8      F
1       1        720       6      F
2       1        971      10      F
3       1       1095       7      F
4       1       1616      10      F
17359346


In [6]:
merged = merged.rename({'gender':'user_gender'},axis=1)

merged.head()

,userid,profileid,rating,user_gender
0,1,133,8,F
1,1,720,6,F
2,1,971,10,F
3,1,1095,7,F
4,1,1616,10,F


In [7]:
merged = merged.merge(gender,left_on='profileid',right_on='userid',how='left')

merged = merged.rename({'gender':'profile_gender'},axis=1)

merged.head()

,userid_x,profileid,rating,user_gender,userid_y,profile_gender
0,1,133,8,F,133,M
1,1,720,6,F,720,F
2,1,971,10,F,971,M
3,1,1095,7,F,1095,M
4,1,1616,10,F,1616,M


In [8]:
len(merged)

17359346

In [9]:
merged = merged.drop('userid_y',axis=1)

merged.head()

,userid_x,profileid,rating,user_gender,profile_gender
0,1,133,8,F,M
1,1,720,6,F,F
2,1,971,10,F,M
3,1,1095,7,F,M
4,1,1616,10,F,M


In [10]:
merged = merged.rename({'userid_x':'userid'},axis=1)

merged.head()

,userid,profileid,rating,user_gender,profile_gender
0,1,133,8,F,M
1,1,720,6,F,F
2,1,971,10,F,M
3,1,1095,7,F,M
4,1,1616,10,F,M


In [11]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(merged, test_size=0.25)

In [12]:
print(len(train_data))
print(len(test_data))

13019509
4339837


In [13]:
train_data.itertuples()

In [14]:
i = 0

for line in train_data.itertuples():
    print(line)
    print(line[0])
    print(line[1])
    print(line[2])
    print(i)
    if i > 5:
        break
    i += 1

Pandas(Index=122490, userid=816, profileid=93458, rating=6, user_gender='F', profile_gender='M')
122490
816
93458
0
Pandas(Index=14679755, userid=114598, profileid=78020, rating=5, user_gender='M', profile_gender='F')
14679755
114598
78020
1
Pandas(Index=7855107, userid=61629, profileid=146192, rating=7, user_gender='F', profile_gender='U')
7855107
61629
146192
2
Pandas(Index=2822509, userid=22623, profileid=145850, rating=1, user_gender='M', profile_gender='U')
2822509
22623
145850
3
Pandas(Index=7007758, userid=55607, profileid=59872, rating=10, user_gender='F', profile_gender='M')
7007758
55607
59872
4
Pandas(Index=6047924, userid=48160, profileid=69603, rating=10, user_gender='F', profile_gender='M')
6047924
48160
69603
5
Pandas(Index=2751792, userid=22143, profileid=127525, rating=2, user_gender='M', profile_gender='F')
2751792
22143
127525
6


In [15]:
n_comment = max(merged.userid)

n_receive = max(merged.profileid)

print(n_comment)
print(n_receive)

135359
220970


In [16]:
from scipy import sparse
from scipy.sparse import coo_matrix

In [17]:
train_data_col = []
train_data_row = []
train_data_value = []

for line in train_data.itertuples():
    train_data_row.append(line[1]-1)
    train_data_col.append(line[2]-1)
    train_data_value.append(line[3])
    
train_data_matrix = sparse.coo_matrix((train_data_value,(train_data_row,train_data_col)))

train_data_matrix

<135359x220970 sparse matrix of type '<class 'numpy.int32'>'
	with 13019509 stored elements in COOrdinate format>

In [18]:
test_data_col = []
test_data_row = []
test_data_value = []

for line in test_data.itertuples():
    test_data_row.append(line[1]-1)
    test_data_col.append(line[2]-1)
    test_data_value.append(line[3])
    
test_data_matrix = sparse.coo_matrix((test_data_value,(test_data_row,test_data_col)))

test_data_matrix

<135359x220970 sparse matrix of type '<class 'numpy.int32'>'
	with 4339837 stored elements in COOrdinate format>

In [19]:
sparsity=round(1.0-len(merged)/float(n_comment*n_receive),3)
print('The sparsity level of Dating Dataset is ' +  str(sparsity*100) + '%')

The sparsity level of Dating Dataset is 99.9%


In [20]:
merged.profileid.nunique()

168791

In [21]:
# train_data_csr = train_data_matrix.tocsr()
# train_data_csr

# test_data_csr = test_data_matrix.tocsr()
# train_data_csr

In [22]:
from scipy.sparse.linalg import svds

train_data_matrix = train_data_matrix.asfptype()
test_data_matrix = test_data_matrix.asfptype()

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix)


In [23]:
u

array([[-6.86792968e-05, -5.41493231e-03,  1.99105814e-03,
         5.70152352e-03, -5.00558228e-05,  6.74854834e-03],
       [-2.55635720e-05, -5.23787886e-04,  2.27608404e-03,
         4.34109276e-03,  2.19218649e-03,  2.60901985e-03],
       [ 1.97925695e-04,  1.39791159e-03,  5.91588667e-04,
         8.76687075e-04,  1.01379649e-03,  6.26342131e-04],
       ...,
       [-4.23395319e-05, -6.09134027e-03,  1.63087029e-03,
         6.33675967e-03,  1.23589032e-03,  7.32829504e-03],
       [ 1.29324039e-05, -1.12718431e-04,  1.27454299e-04,
        -1.44192746e-04,  1.00140130e-05,  2.19288259e-05],
       [ 3.11305845e-03, -3.33089340e-03, -1.49102860e-03,
         1.13260681e-03, -2.45634086e-03,  3.89214879e-03]])

In [24]:
s

array([2140.36496607, 2164.6437929 , 2482.91044001, 2574.67417578,
       3369.28074189, 4331.03336794])

In [25]:
vt

array([[-2.60143913e-04, -8.67534773e-04,  0.00000000e+00, ...,
         7.75916625e-07,  8.05110523e-07,  5.65580196e-05],
       [ 6.28926794e-05,  2.13651571e-03,  0.00000000e+00, ...,
        -3.45915559e-07, -2.87512398e-07, -2.96994237e-05],
       [ 6.16583751e-04,  1.25067322e-03,  0.00000000e+00, ...,
         1.34355473e-06,  1.04102323e-06, -1.72927419e-05],
       [-4.97424406e-04, -2.89032261e-03,  0.00000000e+00, ...,
        -6.90721461e-07, -7.58091550e-07,  8.81441309e-06],
       [-1.45967940e-04, -1.53610936e-03,  0.00000000e+00, ...,
        -6.68491188e-08, -1.41190177e-07, -1.54909920e-05],
       [ 1.87656069e-04,  1.75217256e-03,  0.00000000e+00, ...,
         5.69895605e-08,  1.80641328e-07,  2.43100272e-05]])

In [26]:
u = sparse.coo_matrix(u)
u

<135359x6 sparse matrix of type '<class 'numpy.float64'>'
	with 812154 stored elements in COOrdinate format>

In [27]:
s_diag = sparse.coo_matrix(np.diag(s))
s_diag

<6x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in COOrdinate format>

In [28]:
vt = sparse.coo_matrix(vt)
vt

<6x220970 sparse matrix of type '<class 'numpy.float64'>'
	with 965808 stored elements in COOrdinate format>

In [29]:
u = u.tocsr()
s_diag = s_diag.tocsr()
vt = vt.tocsr()

vt

<6x220970 sparse matrix of type '<class 'numpy.float64'>'
	with 965808 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [31]:
a = np.array([1,2,3,4])
a = sparse.coo_matrix(a)

np.dot(a,a.T)

<1x1 sparse matrix of type '<class 'numpy.intc'>'
	with 1 stored elements in Compressed Sparse Row format>

In [32]:
first_dot = np.dot(u, s_diag)
first_dot

<135359x6 sparse matrix of type '<class 'numpy.float64'>'
	with 812154 stored elements in Compressed Sparse Row format>

In [33]:
test_data_matrix = test_data_matrix.tocsr()
test_data_matrix[0,0]

0.0

In [34]:
test_data_matrix.shape

(135359, 220970)

In [35]:
vt[1,1]

0.002136515707716011

In [36]:
s_diag[1,1]

2164.64379290305

In [37]:
first_dot[1,:].toarray()

array([[-0.05471537, -1.1338142 ,  5.65131284, 11.17689942,  7.38609173,
        11.29975204]])

In [38]:
X_pred_row = []
X_pred_row.append(1)
X_pred_row.append(2)

X_pred_row

[1, 2]

In [39]:
row = test_data_matrix.shape[0]
col = test_data_matrix.shape[1]

In [40]:
max_row = round(row/1000)+1
max_row

136

In [41]:
int_dot = sparse.csr_matrix(([],([],[])),shape=(0,220970))

int_dot

<0x220970 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [42]:
for i in range(0,max_row):
  tmp_dot = first_dot[i*1000:(i+1)*1000,:].dot(vt)
  tmp_dot = tmp_dot.astype(int)
  tmp_dot_array = tmp_dot.toarray()
  tmp_dot = sparse.csr_matrix(tmp_dot_array)
  int_dot = sparse.vstack((int_dot, tmp_dot))
  print(i)
int_dot

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135


<135359x220970 sparse matrix of type '<class 'numpy.float64'>'
	with 7193696 stored elements in Compressed Sparse Row format>

In [43]:
print(row)
print(col)

135359
220970


In [44]:
test_data_matrix[0,0]

0.0

In [45]:
X_pred = int_dot

print('CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

CF MSE: 6.418607407408945


In [55]:
test_data_matrix

<135359x220970 sparse matrix of type '<class 'numpy.float64'>'
	with 4339837 stored elements in Compressed Sparse Row format>

In [56]:
print('Done with ','CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

Done with  CF MSE: 6.418607407408945
